## Plan of Action


1.   We are using **Amazon Alexa Reviews dataset (3150 reviews)**, that contains: **customer reviews, rating out of 5**, date of review, Alexa variant 
2.   First we  **generate sentiment labels: positive/negative**, by marking *positive for reviews with rating >3 and negative for remaining*
3. Then, we **clean dataset through Vectorization Feature Engineering** (TF-IDF) - a popular technique
4. Post that, we use **Support Vector Classifier for Model Fitting** and check for model performance (*we are getting >90% accuracy*)
5. Last, we use our model to do **predictions on real Amazon reviews** using: a simple way and then a fancy way



## Import datasets

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/My Drive/1_LiveProjects/Project6_SentimentAnalysis_with_Pipeline
!ls

In [ ]:
#Loading the dataset
dump = pd.read_csv('a1_AmazonAlexa_ReviewsDataset.tsv',sep='\t') 

dump

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1
...,...,...,...,...,...
3145,5,30-Jul-18,Black Dot,"Perfect for kids, adults and everyone in betwe...",1
3146,5,30-Jul-18,Black Dot,"Listening to music, searching locations, check...",1
3147,5,30-Jul-18,Black Dot,"I do love these things, i have them running my...",1
3148,5,30-Jul-18,White Dot,Only complaint I have is that the sound qualit...,1


## Data Pre-Processing

In [ ]:
dataset = dump[['verified_reviews','rating']]
dataset.columns = ['Review', 'Sentiment']

dataset.head()

,Review,Sentiment
0,Love my Echo!,5
1,Loved it!,5
2,"Sometimes while playing a game, you can answer...",4
3,I have had a lot of fun with this thing. My 4 ...,5
4,Music,5


In [ ]:
# Creating a new column sentiment based on overall ratings
def compute_sentiments(labels):
  sentiments = []
  for label in labels:
    if label > 3.0:
      sentiment = 1
    elif label <= 3.0:
      sentiment = 0
    sentiments.append(sentiment)
  return sentiments

In [ ]:
dataset['Sentiment'] = compute_sentiments(dataset.Sentiment)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
dataset.head()

,Review,Sentiment
0,Love my Echo!,1
1,Loved it!,1
2,"Sometimes while playing a game, you can answer...",1
3,I have had a lot of fun with this thing. My 4 ...,1
4,Music,1


In [ ]:
# check distribution of sentiments
dataset['Sentiment'].value_counts()

1    2741
0     409
Name: Sentiment, dtype: int64

In [ ]:
# check for null values
dataset.isnull().sum()

# no null values in the data

Review       0
Sentiment    0
dtype: int64

### Data Transformation

In [ ]:
x = dataset['Review']
y = dataset['Sentiment']

In [ ]:
# import tokenizer_input
from b2_tokenizer_input import CustomTokenizerExample

In [ ]:
! cat b2_tokenizer_input.py

import spacy
nlp = spacy.load('en_core_web_sm')

import string
punct = string.punctuation
# punct

from spacy.lang.en.stop_words import STOP_WORDS
stopwords = list(STOP_WORDS) # list of stopwords

class CustomTokenizerExample():
    def __init__(self):
        pass

    def text_data_cleaning(self,sentence):
        doc = nlp(sentence)                         # spaCy tokenize text & call doc components, in order

        tokens = [] # list of tokens
        for token in doc:
            if token.lemma_ != "-PRON-":
                temp = token.lemma_.lower().strip()
            else:
              temp = token.lower_
            tokens.append(temp)

        cleaned_tokens = []
        for token in tokens:
            if token not in stopwords and token not in punct:
                cleaned_tokens.append(token)
        return cleaned_tokens

In [ ]:
# if root form of that word is not proper noun then it is going to convert that into lower form
# and if that word is a proper noun, then we are directly taking lower form,
# because there is no lemma for proper noun

# stopwords and punctuations removed

In [ ]:
# let's do a test
token = CustomTokenizerExample()
token.text_data_cleaning("Those were the best days of my life!")

['good', 'day', 'life']

### Feature Engineering (TF-IDF)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(tokenizer=token.text_data_cleaning)
# tokenizer=text_data_cleaning, tokenization will be done according to this function

## Train the model

### Train/ Test Split

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = dataset.Sentiment, random_state = 0)

In [ ]:
x_train.shape, x_test.shape
# 2520 samples in training dataset and 630 in test dataset

((2520,), (630,))

### Fit x_train and y_train

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

In [ ]:
classifier = LinearSVC()

In [ ]:
pipeline = Pipeline([('tfidf',tfidf), ('clf',classifier)])
# it will first do vectorization and then it will do classification

In [ ]:
pipeline.fit(x_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<bound method CustomTokenizerExample.text_data_cleaning of <b2_tokenizer_input.CustomTokenizerExample object at 0x7f710606c6d0>>)),
                ('clf', LinearSVC())])

## Check Model Performance

In [ ]:
y_pred = pipeline.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
# With pipeline, we don't need to prepare the dataset for testing (x_test)

In [ ]:
# confusion_matrix
confusion_matrix(y_test, y_pred)

# we are getting almost 91% accuracy

array([[ 39,  43],
       [ 11, 537]])

In [ ]:
# classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.48      0.59        82
           1       0.93      0.98      0.95       548

    accuracy                           0.91       630
   macro avg       0.85      0.73      0.77       630
weighted avg       0.91      0.91      0.91       630



In [ ]:
# round(accuracy_score(y_test, y_pred)*100,2)

# Model Serialization

In [ ]:
import joblib
joblib.dump(pipeline,'c1_SentimentAnalysis_Model_Pipeline.pkl')

['c1_SentimentAnalysis_Model_Pipeline.pkl']

# Predict Sentiments using Model

### Simple way

In [ ]:
prediction = pipeline.predict(["Alexa is good"])

if prediction == 1:
  print("Result: This review is positive")
else:
  print("Result: This review is negative")

Result: This review is positive


### Fancy way

In [ ]:
new_review = []
pred_sentiment = []

while True:
  
  # ask for a new amazon alexa review
  review = input("Please type an Alexa review - ")

  if review == 'skip':
    print("See you soon!")
    break
  else:
    prediction = pipeline.predict([review])

    if prediction == 1:
      result = 'Positive'
      print("Result: This review is positive\n")
    else:
      result = 'Negative'
      print("Result: This review is negative\n")
  
  new_review.append(review)
  pred_sentiment.append(result)

Please type an Alexa review - alexa is good
Result: This review is positive

Please type an Alexa review - alexa is negative
Result: This review is positive

Please type an Alexa review - alexa is bad
Result: This review is negative

Please type an Alexa review - skip
See you soon!


In [ ]:
Results_Summary = pd.DataFrame(
    {'New Review': new_review,
     'Sentiment': pred_sentiment,
    })

Results_Summary.to_csv("./c2_Predicted_Sentiments.tsv", sep='\t', encoding='UTF-8', index=False)
Results_Summary

,New Review,Sentiment
0,alexa is good,Positive
1,alexa is negative,Positive
2,alexa is bad,Negative


## From Pipeline Pickle

In [ ]:
# model = joblib.load('c1_SentimentAnalysis_Model_Pipeline.pkl')

In [ ]:
# new_review=["bad product"]

In [ ]:
# model.predict(new_review)[0]

# Model Deployment

In [ ]:
! which python

/usr/local/bin/python


In [ ]:
import joblib

joblib.__version__

'1.1.0'